In [2]:
import pandas as pd
import numpy as np

In [3]:
calories9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/calories.json')
distance9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/distance.json')
l_act9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/lightly_active_minutes.json')
m_act9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/moderately_active_minutes.json')
r_heart9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/resting_heart_rate.json')
sedentary9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/sedentary_minutes.json')
sleep_score9 = pd.read_csv('osfstorage-archive/pmdata/p09/fitbit/sleep_score.csv')
sleep9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/sleep.json')
steps9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/steps.json')
hr_zone9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/time_in_heart_rate_zones.json')
v_act9 = pd.read_json('osfstorage-archive/pmdata/p09/fitbit/very_active_minutes.json')

In [4]:
#data frequency to date using sum
calories_processed = calories9.resample('D', on = 'dateTime').value.sum() 
distance_processed = distance9.resample('D', on = 'dateTime').value.sum()
steps_processed = steps9.resample('D', on = 'dateTime').value.sum()

In [5]:
#resting heart rate to simple df
r_heart9['value']=r_heart9['value'].apply(lambda x: x['value'])
r_heart_processed=r_heart9

In [6]:
#delete time in sleep score time stamp
sleep_score9['timestamp']=sleep_score9['timestamp'].apply(lambda x: x[:10])
#drop unnecessary column
sleep_score_processed = sleep_score9.drop(columns='sleep_log_entry_id')

In [7]:
#delete time in sleep score time stamp
sleep_score9['timestamp']=sleep_score9['timestamp'].apply(lambda x: x[:10])
#drop unnecessary column
sleep_score_processed = sleep_score9.drop(columns='sleep_log_entry_id')
#dictionary with heart rate zone minutes to df
hr_zone_value = pd.DataFrame([eval(str(a).replace('{\'valuesInZones\': ','').replace('}}','}')) for a in hr_zone9['value']])
hr_zone_processed = pd.concat([hr_zone9['dateTime'], hr_zone_value], axis=1)

In [8]:
#unify index
l_act_processed = l_act9.set_index('dateTime')
m_act_processed = m_act9.set_index('dateTime')
r_heart_processed = r_heart_processed.set_index('dateTime')
sedentary_processed = sedentary9.set_index('dateTime')
sleep_score_processed = sleep_score_processed.set_index('timestamp')
sleep_processed = sleep9.set_index('dateOfSleep')
hr_zone_processed = hr_zone_processed.set_index('dateTime')
v_act_processed = v_act9.set_index('dateTime')
#index to DateTimeIndex
sleep_score_processed.index = pd.to_datetime(sleep_score_processed.index)

In [9]:
#get dummy values for mainsleep true(1)/false(0)
sleep_processed['mainSleep'] = pd.get_dummies(sleep_processed['mainSleep'])[True].fillna(0)
#drop unnecessary columns
sleep_processed = sleep_processed.drop(['logId','startTime','endTime','type','infoCode','levels'], axis=1)
#index to DateTimeIndex
sleep_processed.index = pd.to_datetime(sleep_processed.index)

In [10]:
#read prediction target
wellness = pd.read_csv('osfstorage-archive/pmdata/p09/pmsys/wellness.csv') 
wellness['effective_time_frame'] = wellness['effective_time_frame'].apply(lambda x: x[:10])
wellness = wellness.rename(columns={"effective_time_frame": "dateTime"})
readiness = wellness.loc[:,['dateTime','readiness']].set_index('dateTime')
readiness.index = pd.to_datetime(readiness.index)

In [11]:
#drop duplicated records
readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed = [x[~x.index.duplicated(keep='first')] for x in [readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed]]

In [14]:
#concat data
P09 = pd.concat([readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed], axis=1)

In [15]:
#rename columns
P09.columns=['readiness','calories','distance','l_act','m_act','resting_heart','sedentary','sleep_overall_score','sleep_composition_score',
             'sleep_revitalization_score', 'sleep_duration_score',
             'deep_sleep_in_minutes', 'sleep_resting_heart_rate','sleep_restlessness',
             'sleep_duration','sleep_minutesToFallAsleep','sleep_minutesAsleep','sleep_minutesAwake',
             'sleep_minutesAfterWakeup','sleep_timeInBed','sleep_efficiency',
             'main_sleep','steps',
             'hr_below_zone1', 'hr_zone1', 'hr_zone3','hr_zone2','v_act']

In [16]:
P09

,readiness,calories,distance,l_act,m_act,resting_heart,sedentary,sleep_overall_score,sleep_composition_score,sleep_revitalization_score,...,sleep_minutesAfterWakeup,sleep_timeInBed,sleep_efficiency,main_sleep,steps,hr_below_zone1,hr_zone1,hr_zone3,hr_zone2,v_act
2019-11-01,NaN,2427.62,300500,121,0,60.983291,931,35.0,14.0,18.0,...,1.0,388.0,93.0,1.0,4014,0.0,895.0,0.0,19.0,0
2019-11-02,NaN,2199.21,206880,68,0,0.000000,1372,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2775,0.0,201.0,0.0,6.0,0
2019-11-03,NaN,3134.91,716360,219,14,61.759336,746,76.0,22.0,19.0,...,10.0,430.0,96.0,1.0,9663,0.0,1163.0,0.0,65.0,31
2019-11-04,NaN,2811.43,457620,199,0,60.836679,874,67.0,17.0,23.0,...,0.0,367.0,96.0,1.0,6140,0.0,928.0,0.0,36.0,0
2019-11-05,NaN,2683.07,421740,163,4,59.332898,834,80.0,19.0,24.0,...,0.0,433.0,97.0,1.0,5704,0.0,1293.0,0.0,25.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,NaN,2080.28,36530,41,0,57.553473,855,75.0,17.0,21.0,...,0.0,544.0,94.0,1.0,492,0.0,1196.0,0.0,10.0,0
2020-03-28,NaN,3040.30,283530,132,49,58.318448,1130,NaN,NaN,NaN,...,0.0,107.0,99.0,1.0,3982,11.0,420.0,63.0,137.0,22
2020-03-29,NaN,2469.98,80210,73,13,60.342356,808,76.0,20.0,17.0,...,0.0,544.0,91.0,1.0,1173,3.0,1330.0,0.0,36.0,2
2020-03-30,4.0,2149.54,79660,67,0,58.196049,748,88.0,23.0,21.0,...,1.0,625.0,96.0,1.0,1070,0.0,1309.0,0.0,4.0,0
